In [25]:
K = 50

In [26]:
import pandas
import glob
import stats
import scipy
import numpy

In [27]:
train = pandas.read_csv('train.csv')

In [28]:
train.head(1)

,project_id,run_accession,name,flag,ref_name,ref_pos,map_quality,cigar,next_ref_name,next_ref_pos,length,seq,qual,tags,seq_len
0,PRJNA281708,SRR1982727,SRR1982727.183616,0,15,40555471,60,101M,*,0,0,AAGTAAATACCAAATTATCAGTGGGTCAGGAGTCTGCATCTTCAAA...,@CCDDFFFHHHHHIIJJJJJIFHCHEDFHJIJFGHIIJJIIIJJJJ...,"['AS:i:0', 'XN:i:0', 'XM:i:0', 'XO:i:0', 'XG:i...",101


In [29]:
test = pandas.read_csv('test.csv')

In [30]:
def clean_seq(sequences, K):
    tmp = []
    nucleotides = [x*K for x in ['A','C','G','T','N']]
    for sequence in sequences:
        if any([True if repeated_nucleotide in sequence else False for repeated_nucleotide in nucleotides ]):
            continue
        else:
            tmp.append(sequence)   
    return tmp

In [31]:
def build_kmers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers
    
def read_kmers_from_list(array, ksize):
  all_kmers = []
  for sequence in array:
    kmers = build_kmers(sequence, ksize)
    all_kmers += kmers
  return all_kmers

In [32]:
def get_location_data(project_ID):
    import requests
    import json
    import pandas
    url = 'https://www.ebi.ac.uk/ena/portal/api/filereport'
    params = {'accession':"{}".format(project_ID),
              'fields':"center_name,instrument_platform,instrument_model,host",
            'format':'json',
            'download':'false',
            'result':'read_run'}
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"+
             " AppleWebKit/537.36 (KHTML, like Gecko)"+
             " Chrome/70.0.3538.77 Safari/537.36"}

    print("Accessing: ", url)
    r = requests.get(url,
                  params = params,
                  headers = headers
                  ) 
    print("\tMaking Request To: ", r.url) 
    my_json = json.loads(r.text)
    df = pandas.json_normalize(json.loads(r.text))
    df = df.assign(project_id=project_ID)
    
    # Temp Fix if center_name field is empty
    df['center_name'] = df['center_name'].replace('', project_ID)
    return df

In [33]:
#projects  = glob.glob("{}/*".format(bam_files_path), recursive = True)
#projects = [x.split("/")[-1] for x in projects]
projects = ['PRJNA277616','PRJNA281708', 'PRJNA634526']
print(projects)

['PRJNA277616', 'PRJNA281708', 'PRJNA634526']


In [34]:
df = pandas.concat([get_location_data(x) for x in projects], ignore_index=True)
df

Accessing:  https://www.ebi.ac.uk/ena/portal/api/filereport
	Making Request To:  https://www.ebi.ac.uk/ena/portal/api/filereport?accession=PRJNA277616&fields=center_name%2Cinstrument_platform%2Cinstrument_model%2Chost&format=json&download=false&result=read_run
Accessing:  https://www.ebi.ac.uk/ena/portal/api/filereport
	Making Request To:  https://www.ebi.ac.uk/ena/portal/api/filereport?accession=PRJNA281708&fields=center_name%2Cinstrument_platform%2Cinstrument_model%2Chost&format=json&download=false&result=read_run
Accessing:  https://www.ebi.ac.uk/ena/portal/api/filereport
	Making Request To:  https://www.ebi.ac.uk/ena/portal/api/filereport?accession=PRJNA634526&fields=center_name%2Cinstrument_platform%2Cinstrument_model%2Chost&format=json&download=false&result=read_run


,run_accession,sample_accession,center_name,instrument_platform,instrument_model,host,project_id
0,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,,PRJNA277616
1,SRR1909613,SAMN03394592,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,,PRJNA277616
2,SRR1909637,SAMN03394591,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,,PRJNA277616
3,SRR1909638,SAMN03394590,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,,PRJNA277616
4,SRR1909639,SAMN03394589,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,,PRJNA277616
...,...,...,...,...,...,...,...
416,SRR14209704,SAMN18699854,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens,PRJNA634526
417,SRR14209705,SAMN18699853,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens,PRJNA634526
418,SRR14209706,SAMN18699852,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens,PRJNA634526
419,SRR14209707,SAMN18699851,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens,PRJNA634526


In [35]:
train.head(3)

,project_id,run_accession,name,flag,ref_name,ref_pos,map_quality,cigar,next_ref_name,next_ref_pos,length,seq,qual,tags,seq_len
0,PRJNA281708,SRR1982727,SRR1982727.183616,0,15,40555471,60,101M,*,0,0,AAGTAAATACCAAATTATCAGTGGGTCAGGAGTCTGCATCTTCAAA...,@CCDDFFFHHHHHIIJJJJJIFHCHEDFHJIJFGHIIJJIIIJJJJ...,"['AS:i:0', 'XN:i:0', 'XM:i:0', 'XO:i:0', 'XG:i...",101
1,PRJNA277616,SRR1909638,SRR1909638.13403218,133,MT,2920,0,*,=,2920,0,CTCGTGGAGCCATTCATACAGGTCCCTATGGAGCTTTAATTTATTA...,FFFFFBFBFFFFIIFFFBFFFBFFFFFFFFIBFFFFIIFFIIIIFF...,['YT:Z:UP'],98
2,PRJNA281708,SRR1982826,SRR1982826.8793344,0,MT,2225,60,101M,*,0,0,CTAAAAAATCCCAAACATATAACTGAACTCCTCACACCCAATTGGA...,CBBCCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFGGGGGGGGGGG...,"['AS:i:0', 'XN:i:0', 'XM:i:0', 'XO:i:0', 'XG:i...",101


In [36]:
train = train.set_index(['project_id', 'run_accession'])
train.groupby(level=['project_id','run_accession']).size()

project_id   run_accession
PRJNA277616  SRR1867792       3399
             SRR1909613       3373
             SRR1909637       3423
             SRR1909638       3372
             SRR1909639       3350
             SRR1926132       3378
             SRR1926133       3429
             SRR1926134       3318
             SRR1926135       3418
             SRR1926136       3396
PRJNA281708  SRR1982584       3464
             SRR1982585       3460
             SRR1982588       3379
             SRR1982612       3391
             SRR1982660       3402
             SRR1982727       3337
             SRR1982783       3330
             SRR1982787       3423
             SRR1982814       3359
             SRR1982826       3481
PRJNA634526  SRR14209589      3796
             SRR14209591      3767
             SRR14209592      3737
             SRR14209603      3770
             SRR14209614      3778
             SRR14209634      3779
             SRR14209651      3688
             SRR14209655    

In [37]:
train = train.sample(1000)

In [38]:
train = train.sort_index()
indexes = sorted(list(set(train.index)))
mydict = {}
N = len(indexes)
for i in range(N):
    key = indexes[i]
    seq = train.loc[key]['seq'].to_list()
    seq = clean_seq(seq, K)
    arr = read_kmers_from_list(seq, K)
    mydict[key] = arr

In [39]:
tmp = []
for key in mydict.keys():
    mydf = pandas.DataFrame()
    mydf['kmer_seq'] = mydict[key]
    mydf = mydf.assign(project_id=key[0])
    mydf = mydf.assign(run_accession=key[1])
    mydf = pandas.merge(mydf, df, on=['project_id','run_accession'], how='left')
    tmp.append(mydf)
kmer_df  = pandas.concat(tmp, ignore_index=True)

In [40]:
kmer_df

,kmer_seq,project_id,run_accession,sample_accession,center_name,instrument_platform,instrument_model,host
0,GCATGATTTCCCATTCAGAGGCAGGTGCTGCCCTCATATCAGAAAA...,PRJNA277616,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,
1,CATGATTTCCCATTCAGAGGCAGGTGCTGCCCTCATATCAGAAAAG...,PRJNA277616,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,
2,ATGATTTCCCATTCAGAGGCAGGTGCTGCCCTCATATCAGAAAAGT...,PRJNA277616,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,
3,TGATTTCCCATTCAGAGGCAGGTGCTGCCCTCATATCAGAAAAGTA...,PRJNA277616,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,
4,GATTTCCCATTCAGAGGCAGGTGCTGCCCTCATATCAGAAAAGTAG...,PRJNA277616,SRR1867792,SAMN03394583,Weill Cornell Medical College in Qatar,ILLUMINA,Illumina HiSeq 2500,
...,...,...,...,...,...,...,...,...
62193,CGTGTGTGTGTGTGTGTGTGTGGGTGTGTGTGTGTGTGTGTGTGTG...,PRJNA634526,SRR14209708,SAMN18699850,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens
62194,GTGTGTGTGTGTGTGTGTGTGGGTGTGTGTGTGTGTGTGTGTGTGT...,PRJNA634526,SRR14209708,SAMN18699850,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens
62195,TGTGTGTGTGTGTGTGTGTGGGTGTGTGTGTGTGTGTGTGTGTGTG...,PRJNA634526,SRR14209708,SAMN18699850,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens
62196,GTGTGTGTGTGTGTGTGTGGGTGTGTGTGTGTGTGTGTGTGTGTGT...,PRJNA634526,SRR14209708,SAMN18699850,SUB9454898,ILLUMINA,Illumina NovaSeq 6000,Homo sapiens


In [41]:
crosstab = pandas.crosstab(kmer_df.kmer_seq, kmer_df.project_id,margins = True)

In [42]:
crosstab

project_id,PRJNA277616,PRJNA281708,PRJNA634526,All
kmer_seq,,,,
AAAAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAA,1,0,0,1
AAAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAAC,1,0,0,1
AAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACA,1,0,0,1
AAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACAA,1,0,0,1
AAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACAAA,1,0,0,1
...,...,...,...,...
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTTTT,1,0,0,1
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTTT,1,0,0,1
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTT,1,0,0,1


In [43]:
def cross_chi_table(crosstab, row, total, i):

    total = total.iloc[0]
    matrix = numpy.array([[row[0],total[0]-row[0]],
                          [row[1],total[1]-row[1]]])
    try:
        chi2, p, dof, expected = chi2_contingency(matrix)
        crosstab.loc[row.name,"x_y"] = p
    except:
        crosstab.loc[row.name,"x_y"] = numpy.NAN
        
    matrix = numpy.array([[row[1],total[1]-row[1]],
                          [row[2],total[2]-row[2]]])
    try:
        chi2, p, dof, expected = chi2_contingency(matrix)
        crosstab.loc[row.name,"y_z"] = p
    except:
        crosstab.loc[row.name,"y_z"] = numpy.NAN

    matrix = numpy.array([[row[0],total[0]-row[0]],
                          [row[2],total[2]-row[2]]])
    try:
        chi2, p, dof, expected = chi2_contingency(matrix)
        crosstab.loc[row.name,"x_z"] = p
    except:
        crosstab.loc[row.name,"x_z"] = numpy.NAN
    
    return crosstab
crosstab["x_y"] = 0  
crosstab["y_z"] = 0
crosstab["x_z"] = 0
for i in range(crosstab.shape[0]):
    crosstab = cross_chi_table(crosstab,crosstab.iloc[i],crosstab.loc[["All"]],i) 

In [44]:
crosstab

project_id,PRJNA277616,PRJNA281708,PRJNA634526,All,x_y,y_z,x_z
kmer_seq,,,,,,,
AAAAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAA,1,0,0,1,NaN,NaN,NaN
AAAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAAC,1,0,0,1,NaN,NaN,NaN
AAAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACA,1,0,0,1,NaN,NaN,NaN
AAAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACAA,1,0,0,1,NaN,NaN,NaN
AAAAAAAAAAAAAAAAAAAAAACCCAAAAAAAAAACNCTCTAAGAACAAA,1,0,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTTTT,1,0,0,1,NaN,NaN,NaN
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTTT,1,0,0,1,NaN,NaN,NaN
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCACTTTTT,1,0,0,1,NaN,NaN,NaN


In [45]:
p_val_df = crosstab[(crosstab.x_y < 0.05) | (crosstab.y_z < 0.05) | (crosstab.x_z < 0.05)].\
    sort_values(by=['All'], ascending=False)

In [46]:
library = p_val_df.index.to_list()

In [47]:
with open("K_{}_library.txt".format(K), 'w') as file:
    file.write('# K={}\n'.format(K))
    file.write('\n'.join(library))

In [48]:
import sys
sys.exit()

SystemExit: 

/apps/software/standard/core/anaconda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#https://pythonfordatascienceorg.wordpress.com/chi-square-python/
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html?highlight=chi2_contingency#scipy.stats.chi2_contingency
from scipy.stats import chi2_contingency
chi2_contingency(crosstab)

In [ ]:
chi2, p, dof, ex = chi2_contingency(crosstab)

In [ ]:
print("K: ",K)
print("Chi-Square: ", chi2)
print("p: ", p)

In [ ]:
"""
K = 30
(1547445.6979213262,
 0.0,
 1138764,
 array([[0.28129705, 0.28784497, 0.43085798],
        [0.28129705, 0.28784497, 0.43085798],
        [0.5625941 , 0.57568993, 0.86171597],
        ...,
        [0.28129705, 0.28784497, 0.43085798],
        [1.40648525, 1.43922483, 2.15428992],
        [0.5625941 , 0.57568993, 0.86171597]]))
"""

In [ ]:
"""
K = 55
(1113199.1051552424,
 0.0,
 911602,
 array([[0.25997437, 0.26203298, 0.47799266],
        [0.25997437, 0.26203298, 0.47799266],
        [0.25997437, 0.26203298, 0.47799266],
        ...,
        [0.51994873, 0.52406595, 0.95598531],
        [0.51994873, 0.52406595, 0.95598531],
        [0.51994873, 0.52406595, 0.95598531]]))
"""

In [ ]:
"""
K = 80
(652239.615902771,
 0.0,
 582422,
 array([[0.20614538, 0.21375238, 0.58010224],
        [0.20614538, 0.21375238, 0.58010224],
        [0.20614538, 0.21375238, 0.58010224],
        ...,
        [0.20614538, 0.21375238, 0.58010224],
        [0.20614538, 0.21375238, 0.58010224],
        [0.20614538, 0.21375238, 0.58010224]]))
"""